In [ ]:
!pip install google-cloud-pubsub

In [ ]:
from google.cloud import pubsub_v1
import os
import pandas as pd
from json import dumps
import logging

# Provide the path to the service account key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'YOUR CREDENTIAL GCP'

# Define your project ID and the topic name
project_id = "YOUR PROJECT ID"
topic_name = "stock_market_real_time_pub_sub"

# Load your data into a DataFrame
df = pd.read_csv('indexProcessed.csv')

# Initialize the Pub/Sub Publisher client
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_name)

# Infinite loop to continuously generate and publish events
while True:
    try:
        # Randomly sample one record from the DataFrame and convert it to a dictionary
        dict_stock = df.sample(1).to_dict(orient='records')[0]
        serialized_data = dumps(dict_stock)
        print(serialized_data)

        # Data must be a bytestring
        data = serialized_data.encode('utf-8')

        # When you publish a message, the client returns a future.
        future = publisher.publish(topic_path, data, event_type='stock_update')
        logging.info(f"Record sent successfully, Message ID: {future.result()}")

    except Exception as e:
        logging.error(f"Failed to send record: {str(e)}")